# AymaraSDK + OpenAI Minimal Example

This notebook demonstrates:
- Creating an eval with AymaraSDK
- Fetching eval prompts
- Calling OpenAI (real API) with those prompts
- Creating an eval run with the responses

## Requirements
- Set `OPENAI_API_KEY` and `AYMARA_SDK_API_KEY` in your environment or `.env` file.
- Install dependencies: `pip install openai aymara-sdk-python python-dotenv`

In [1]:
# Environment and imports
import os
from dotenv import load_dotenv
load_dotenv()

import openai
from aymara_sdk import AymaraSDK
from aymara_sdk.types.eval_run_create_params import Response

## Set up API keys

In [2]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise RuntimeError("OPENAI_API_KEY not set in environment.")
openai.api_key = OPENAI_API_KEY

## Instantiate the AymaraSDK client

In [3]:
client = AymaraSDK()

## Create an eval

In [ ]:
eval_obj = client.evals.create(
    ai_description="Minimal SDK Example Eval",
    ai_instructions="Answer the prompts as best as you can.",
    eval_type="safety",
    name="minimal-example-eval",
    num_prompts=5
)
eval_id = eval_obj.eval_uuid
if not eval_id:
    raise RuntimeError("Eval creation failed.")
print(f"Created eval: {eval_id}")

Created eval: test.4bd76ad0-129f-485a-bc2d-e2d227f644d8


## Fetch prompts for the eval

In [ ]:
prompts_response = client.evals.get_prompts(eval_id)
prompts = prompts_response.items
if not prompts:
    raise RuntimeError("No prompts found for eval.")
print(f"Fetched {len(prompts)} prompts.")

Fetched 50 prompts.


## Call OpenAI for each prompt and collect responses

In [ ]:
responses: list[Response] = []
for prompt in prompts:
    prompt_text = prompt.content
    prompt_uuid = prompt.prompt_uuid
    if not prompt_uuid:
        print("Prompt missing prompt_uuid, skipping.")
        continue
    try:
        completion = openai.completions.create(
            model="gpt-4.1-nano-2025-04-14",
            prompt=[ prompt_text],
            max_tokens=256,
            temperature=0.7,
        )
        answer = completion.choices[0].text.strip()
    except Exception as e:
        print(f"OpenAI call failed for prompt '{prompt_text}': {e}")
        answer = "ERROR"
    responses.append(Response(
        content=answer,
        prompt_uuid=prompt_uuid
    ))

## Create an eval run with the responses

In [ ]:
eval_run = client.eval_runs.create(
    eval_uuid=eval_id,
    responses=responses
)
eval_run_id = getattr(eval_run, "eval_run_uuid", None)
print(f"Eval run created: {eval_run_id}")